In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

WE ARE GOING TO LOAD THE DATASET

In [ ]:
cardf = pd.read_csv('../input/cardata/car data.csv')

In [ ]:
%config Completer.use_jedi = False

WE ARE PRINTING JUST THE FIRST 10 ROWS

In [ ]:
cardf.head()

In [ ]:
cardf.shape

CHECKING ALL UNIQUE VALUES

In [ ]:
cols = cardf.columns
print(cols)

In [ ]:
for i in cols:
    print(cardf[i].unique())

checking missing values in our dataset

In [ ]:
cardf.isnull().sum()

As we can see that there are no null values in our dataset so it is good for us 

AS WE DONT NEED THE CAR NAME BECAUSE THERE CAN BE MULTIPLE AND WE DONT HAVE SUCH USAGE OF IT RIGHT NOW

In [ ]:
new_cardf = cardf[['Year','Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]

In [ ]:
new_cardf.head()

NOW WE WILL CREATE ANOTHER COLUMN WITH THE  CURRENT YEAR  THAT WILL HELP US IN FINDING OUT HOW OLD THE CAR IS

In [ ]:
from datetime import date
today = date.today()
new_cardf['Current_Year'] = today.year

In [ ]:
new_cardf.head(2)

**> *now to create a feautre of how old the car is***

In [ ]:
new_cardf['Cars_age'] = new_cardf['Current_Year'] - new_cardf['Year']

In [ ]:
new_cardf.head(2)

> **now as we have calcualted the age of the car , we can now drop year and current year columns from our dataset**

In [ ]:
new_cardf.drop(['Current_Year','Year'], axis=1 , inplace=True)

In [ ]:
new_cardf.head(2)

> **NOW ITS TIME TO CONVERT THE CATEGORICAL VALUES INTO OTHER VALUES FOR EX FUEL TYPE TO AN INTEGER VALUE , BY USING GET DUMMIES INSIDE PANDAS IN ONE HOT ENCODING**

> **we use drop first to del 1st column to protect it from dummy variable tap(google about it)**

In [ ]:
new_cardf = pd.get_dummies(new_cardf, drop_first =True)

In [ ]:
new_cardf.head()

NOW WE WILL CHECK FOR CORRELATIONS BETWEEN THE COLUMNS OF OUR DATASET (-1 to 1) more correlated as goes near to 1

In [ ]:
    new_cardf.corr()

now lets visualize the above correlations

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.pairplot(new_cardf)

In [ ]:
corrit = new_cardf.corr()
tops_features = corrit.index
plt.figure(figsize=(30,30))
fig = sns.heatmap(new_cardf[tops_features].corr() , annot = True, cmap='RdYlGn')


In [ ]:
X = new_cardf.iloc[:,1:]
y = new_cardf.iloc[:,0]

In [ ]:
print(" X " , X.head())
print()
print(" y ", y.head())

Important Features

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
print(model.feature_importances_)

now lets plot the top most important features

In [ ]:
top_imp = pd.Series(model.feature_importances_, index= X.columns)
top_imp.nlargest(5).plot(kind= 'barh')
plt.show()

now lets train the data set , first lets split it in train and test data

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size= 0.2, random_state = 42)

In [ ]:
X_train.shape


Now we will use random forest regressor 

In [ ]:
from sklearn.ensemble import RandomForestRegressor


In [ ]:
n_estimators = [int(x) for x in np.linspace(start= 100, stop = 1300, num=12)]
max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
min_samples_split = [3, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10]

set hyper parameters 

In [ ]:
random_est = {'n_estimators':n_estimators ,
              'max_features':max_features,
              'max_depth':max_depth,
              'min_samples_split':min_samples_split,
              'min_samples_leaf':min_samples_leaf
             }

we will use the random grid to search for best hyperparameters

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
rf = RandomForestRegressor()

In [ ]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_est,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

> **this will give best parameters**

In [ ]:
rf_random.fit(X_train , y_train)

In [ ]:
rf_random.best_params_

In [ ]:
pred = rf_random.predict(X_test)
print(pred)

In [ ]:
sns.distplot(y_test-pred)

In [ ]:
plt.scatter(y_test,pred)


In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, pred))
print('MSE:', metrics.mean_squared_error(y_test, pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred)))